In [1]:
import numba
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.optimize import minimize
from scipy.optimize import minimize
from scipy.special import gammaln

import healpy as hp
from astropy.table import Table
from astropy.io import fits
from astropy import units as u
from astropy.coordinates import SkyCoord as SkyCoord

In [8]:
# Elimando fonte que estejam com B em [-b,b]
def galactic_cut(data,RA_column_index,Dec_column_index,Lim_inf,Lim_sup):

    data_array = np.array(data)

    ra_column_array = data_array[:,RA_column_index]
    dec_column_array = data_array[:,Dec_column_index]

    c = SkyCoord(ra_column_array,dec_column_array, frame='icrs', unit='deg')

    b_array = np.array(c.galactic.b)

    Index = 0
    Index_list = []
    for b in b_array:
        if b > Lim_inf  and b < Lim_sup:
            Index_list.append(Index)
        Index+=1

    index_to_delete = np.array(Index_list)
    new_data_array = np.delete(data_array,index_to_delete,axis=0)
    new_data = pd.DataFrame(new_data_array,columns=data.columns.values)

    return new_data

def exclui_pixels_vazios(density_map,thetas_pixels,phis_pixels):
    
    indices_para_excluir = np.where(density_map==0)[0]
    density_map = np.delete(density_map,indices_para_excluir)

    thetas_pixels = np.delete(thetas_pixels,indices_para_excluir)
    phis_pixels = np.delete(phis_pixels,indices_para_excluir)

    n_x = np.sin(thetas_pixels)*np.cos(phis_pixels)
    n_y = np.sin(thetas_pixels)*np.sin(phis_pixels)
    n_z = np.cos(thetas_pixels)

    n_vec = np.stack([n_x,n_y,n_z],axis=1)

    return density_map, n_vec

# Estimador likelihood
def negativo_likelihood(params):
    
    M = params[0]
    d_vec = params[1:]
    
    lambdas = M*(1+np.dot(n_vec,d_vec))

    if np.any(lambdas <= 0):
        return 1e10

    logP = Ni*np.log(lambdas)-lambdas - gammaln(Ni+1)

    return -np.sum(logP)

def dipolo_log_likelihood(M,d_vec):
    print('') 
    print('Maximizando L')
    initial_guess = [M, d_vec[0], d_vec[1], d_vec[2]]

    result = minimize(negativo_likelihood, initial_guess, method='BFGS')

    M0_best = result.x[0]
    Dx_best = result.x[1]
    Dy_best = result.x[2]
    Dz_best = result.x[3]

    D = np.sqrt( (Dx_best**2)+(Dy_best**2)+(Dz_best**2))/M0_best

    D_modulo = np.sqrt(Dx_best**2 + Dy_best**2 + Dz_best**2)

    n_x_d = Dx_best / D_modulo
    n_y_d = Dy_best / D_modulo
    n_z_d = Dz_best / D_modulo

    n = np.array([n_x_d, n_y_d, n_z_d])
    theta_n, phi_n = hp.vec2ang(n)
    l = np.degrees(phi_n)
    b = 90 - np.degrees(theta_n)

    l,b = l[0],b[0]

    Coord = SkyCoord(l,b, frame='galactic', unit='deg')
    ra, dec = Coord.icrs.ra.value, Coord.icrs.dec.value

    print('Rodando uma única vez (Método BFGS):\n')

    print(f'D=|d|/M0={D:.4f}, |vec(d)| = {D_modulo:.4f}'), 
    print(f'(l,b) = ({l:.2f}°,{b:.2f}°)')
    print(f'(RA,DEC) = ({ra:.2f}°,{dec:.2f}°)')

    return [[D], [D_modulo], [l,b], [ra,dec]]

# Estimador χ2
def chi2(params):
    
    M0 = params[0]
    d_vec = params[1:]
            
    N_i_model = M0 + np.dot(n_vec,d_vec)
            
    chi2_value = np.sum((Ni - N_i_model)**2 / Ni)
            
    return chi2_value

def dipolo_chi2(M,d_vec):
    print('') 
    print('Minimizando χ2')
    initial_guess = [M, d_vec[0], d_vec[1], d_vec[2]]

    print('Rodando uma única vez (BFGS)')
    result = minimize(chi2, initial_guess, method='BFGS')

    M0_best, Dx_best, Dy_best, Dz_best = result.x[0], result.x[1], result.x[2], result.x[3]

    # anisotropia_dipolar:
    D = np.sqrt( (Dx_best**2)+(Dy_best**2)+(Dz_best**2))/M0_best

    D_modulo = np.sqrt(Dx_best**2 + Dy_best**2 + Dz_best**2)

    cov_matriz = result.hess_inv # Obtendo a inversa da matriz hessiana
    erros = np.sqrt(np.diag(cov_matriz)) # a raiz da diagonal dá os desvios padroes dos 4 parametros

    sigma_M , sigma_dx, sigma_dy, sigma_dz = erros[0], erros[1], erros[2], erros[3] # Erros individuais

    #Assumindo que M e o vec(d) não são correlacionados

    # Propagação do erro de |d|
    sigma_D_modulo = np.sqrt((Dx_best / D_modulo)**2 * sigma_dx**2 + (Dy_best / D_modulo)**2 * sigma_dy**2 + (Dz_best / D_modulo)**2 * sigma_dz**2)

    # Propagação do erro de D = |d| / M
    sigma_D = D * np.sqrt((sigma_D_modulo / D_modulo)**2 +(sigma_M / M)**2)
    
    print('')
    print(f"módulo de d: {D_modulo:.2f} ± {sigma_D_modulo:.2f}")
    print(f"D = |d|/M:   {D:.3f} ± {sigma_D:.3f}")

    n_x_d = Dx_best / D_modulo
    n_y_d = Dy_best / D_modulo
    n_z_d = Dz_best / D_modulo

    n = np.array([n_x_d, n_y_d, n_z_d])
    theta_n, phi_n = hp.vec2ang(n)
    l, b  = np.degrees(phi_n), 90 - np.degrees(theta_n)

    Coord = SkyCoord(l,b, frame='galactic', unit='deg')
    ra, dec = Coord.icrs.ra.value, Coord.icrs.dec.value

    print('')
    print(f'(l,b) = ({l[0]:.2f}°,{b[0]:.2f}°)')
    print(f'(RA,DEC) = ({ra[0]:.2f}°,{dec[0]:.2f}°)')

    return [[D,sigma_D], [D_modulo, sigma_D_modulo], [l,b], [ra,dec]], result

In [3]:
name = r' Densidade de fontes Quaia $G<20.5$'
filename = 'quaia_G20.5.fits'
hdu = fits.open(filename)
hi = hdu[1].data
hi = Table(hi)
hi = hi.to_pandas()

# Cortando |b|<20

b = hi['b']
cut_b = 20

mask = np.where((-cut_b > b) | (b > cut_b))[0]
hi = hi.loc[mask]

data = hi 
nside = 64
npix = hp.nside2npix(nside)
# Obtendo os angulos dos pixels em coordenadas esféricas padrão em radianos
thetas_pixels, phis_pixels = hp.pix2ang(nside, np.arange(npix))
# obtendo as coordenadas centrais cartesianas de cada pixel
central_pixels_positions = hp.ang2vec(thetas_pixels,phis_pixels)

# obtendo as coordenadas cartesianas de cada fontes
l, b = data['l'], data['b']
phis_fontes, thetas_fontes = np.radians(l), np.radians(90.0 - b)
coord_ca_cartesian_array = hp.ang2vec(thetas_fontes,phis_fontes)

pix_indices = hp.ang2pix(nside, thetas_fontes, phis_fontes)
# Contando o número de fontes em cada pixel
density_map = np.bincount(pix_indices, minlength=npix)

In [4]:
# Primeiro estou assumindo: 
M, d_vec = np.mean(density_map), np.array([0.01, 0.01, 0.01])
initial_guess = [M, d_vec[0], d_vec[1], d_vec[2]]

In [9]:
Ni, n_vec = exclui_pixels_vazios(density_map,thetas_pixels,phis_pixels)

resultado1 = dipolo_log_likelihood(M,d_vec)

resultado2 = dipolo_chi2(M,d_vec)


Maximizando L
Rodando uma única vez (Método BFGS):

D=|d|/M0=0.0015, |vec(d)| = 0.0521
(l,b) = (212.84°,42.14°)
(RA,DEC) = (141.68°,17.88°)

Minimizando χ2
Rodando uma única vez (BFGS)

módulo de d: 1.80 ± 0.03
D = |d|/M:   0.067 ± 0.001

(l,b) = (298.15°,66.03°)
(RA,DEC) = (190.92°,3.23°)
